In [14]:
import pandas as pd
import polars as pl
import subprocess

In [12]:
%ls /home/jovyan/data/

'Cuenta Corriente PESOS 16-07-25.xlsx'*
'Cuenta Corriente PESOS 30-03-25.xlsx'*
'Cuenta Corriente PESOS 30-06-25.xlsx'*


In [15]:
# hago un ls de /home/jovyan/data/, me quedo todos los nomrbes de los archivos en un lista
cmd = "ls /home/jovyan/data/"
result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
file_names = result.stdout.splitlines()
print(file_names)

['Cuenta Corriente PESOS 16-07-25.xlsx', 'Cuenta Corriente PESOS 30-03-25.xlsx', 'Cuenta Corriente PESOS 30-06-25.xlsx']


In [17]:
import re

# ordeno ['Cuenta Corriente PESOS 16-07-25.xlsx', 'Cuenta Corriente PESOS 30-03-25.xlsx', 'Cuenta Corriente PESOS 30-06-25.xlsx'] por fecha
def extract_date(filename):
	match = re.search(r'(\d{2}-\d{2}-\d{2})', filename)
	if match:
		return pd.to_datetime(match.group(1), format='%d-%m-%y')
	else:
		return pd.NaT

file_names.sort(key=extract_date)
print(file_names)

['Cuenta Corriente PESOS 30-03-25.xlsx', 'Cuenta Corriente PESOS 30-06-25.xlsx', 'Cuenta Corriente PESOS 16-07-25.xlsx']


In [24]:
min_date = '1900-01-01'
# genero dataframe vacio con el siguiente esquema>
# Liquida        datetime64[ns]
# Operado        datetime64[ns]
# Comprobante            object
# Numero                  int64
# Cantidad              float64
# Especie                object
# Precio                float64
# Importe               float64
# Saldo                 float64
# Referencia             object
df_cuenta_corriente_historic = pd.DataFrame(columns=[
    'Liquida', 'Operado', 'Comprobante', 'Numero', 
    'Cantidad', 'Especie', 'Precio', 'Importe', 
    'Saldo', 'Referencia'
])

for file_name in file_names:
    data_dd_mm_yy = file_name.replace('Cuenta Corriente PESOS ', '').replace('.xlsx', '')
    df_cuenta_corriente = pd.read_excel(f'/home/jovyan/data/{file_name}', sheet_name=file_name.replace('.xlsx', ''))

    # filtro el df para quedarme con las fechas mayores a min_date
    df_cuenta_corriente['Operado'] = pd.to_datetime(df_cuenta_corriente['Operado'], format='%d/%m/%Y', errors='coerce')
    df_cuenta_corriente = df_cuenta_corriente[df_cuenta_corriente['Operado'] > min_date]
    # hago un union con el df_cuenta_corriente_historic
    df_cuenta_corriente_historic = pd.concat([df_cuenta_corriente_historic, df_cuenta_corriente], ignore_index=True)

    # el min_date se actualiza para que sea la maxima fecha de la columna Operado
    if not df_cuenta_corriente.empty:
        min_date = df_cuenta_corriente['Operado'].max()
    print(f'Procesado {file_name} con {len(df_cuenta_corriente)} filas')

df_cuenta_corriente_historic

Procesado Cuenta Corriente PESOS 30-03-25.xlsx con 185 filas
Procesado Cuenta Corriente PESOS 30-06-25.xlsx con 59 filas
Procesado Cuenta Corriente PESOS 16-07-25.xlsx con 17 filas


/tmp/ipykernel_254/1214895697.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_cuenta_corriente_historic = pd.concat([df_cuenta_corriente_historic, df_cuenta_corriente], ignore_index=True)


,Liquida,Operado,Comprobante,Numero,Cantidad,Especie,Precio,Importe,Saldo,Referencia
0,2024-04-09,2024-04-09,RECIBO DE COBRO,546744,0.00,NaN,0.00,"100,000.00","100,124.47",CREDITO CTA. CTE.
1,2024-04-09,2024-04-09,RECIBO DE COBRO,551326,0.00,NaN,0.00,200.00,"100,324.47",CREDITO CTA. CTE.
2,2024-04-10,2024-04-10,SUSCRIPCION FCI,120288,0.00,BULMAAA,0.00,"-80,000.00","20,324.47",NaN
3,2024-04-11,2024-04-09,VENTA,2910915,-10.00,ERIC,"2,662.18","26,621.84","46,946.31",NaN
4,2024-04-11,2024-04-09,COMPRA NORMAL,2918934,20.00,BPAT,"1,298.05","-25,960.92","20,985.39",NaN
...,...,...,...,...,...,...,...,...,...,...
256,2025-07-11,2025-07-01,VENTA CAUCION TERMINO,5062031,-100.00,VARIAS,755.35,"75,535.45","141,074.15",NaN
257,2025-07-11,2025-07-11,RETENCION,994819,0.00,NaN,0.00,-152.52,"140,921.63",RET GGAL
258,2025-07-11,2025-07-11,DIVIDENDOS,966184,0.00,GGAL,0.00,"2,149.84","143,071.47",GGAL BYMA
259,2025-07-14,2025-07-11,COMPRA NORMAL,5377333,250.00,FIPL,259.03,"-64,757.55","78,313.92",NaN


In [25]:
df_cuenta_corriente_historic.to_csv('/home/jovyan/data/cuenta_corriente_historic.csv', index=False)

In [2]:
import pandas as pd

def calculat_profit_and_loss(ruta_archivo_csv: str):
    """
    Calcula la ganancia o pérdida realizada para cada operación de venta
    de activos a partir de un archivo CSV de cuenta corriente.

    Args:
        ruta_archivo_csv (str): La ruta al archivo CSV exportado del broker.

    Returns:
        pandas.DataFrame: Un DataFrame con el resultado de cada venta.
    """
    # 1. Cargar y preparar los datos del CSV
    df = pd.read_csv(
        ruta_archivo_csv,
        delimiter=',',  # El delimitador parece ser punto y coma
        decimal='.'     # Usar coma como separador decimal
    )

    # Convertir columnas a los tipos de datos correctos
    df['Operado'] = pd.to_datetime(df['Operado'], format='mixed', dayfirst=True, errors='coerce')
    # Asegurarse que las columnas numéricas sean floats
    for col in ['Cantidad', 'Precio', 'Importe']:
        if df[col].dtype == 'object':
            df[col] = df[col].str.replace('.', '', regex=False).str.replace(',', '.', regex=False).astype(float)

    # 2. Filtrar solo operaciones de compra y venta
    operaciones = df[df['Comprobante'].isin(['COMPRA NORMAL', 'VENTA'])].copy()
    operaciones = operaciones.sort_values(by='Operado', ascending=True)

    # 3. Lógica principal: Iterar y calcular
    cartera = {}  # Diccionario para seguir el estado de cada activo
                    # Ejemplo: {'GGAL': {'cantidad': 100, 'costo_total': 45000}}
    resultados = [] # Lista para guardar los resultados de las ventas

    print("Procesando operaciones...")

    for index, op in operaciones.iterrows():
        especie = op['Especie']
        cantidad = op['Cantidad']
        importe = op['Importe']
        precio_op = op['Precio']

        # Inicializar el activo en la cartera si no existe
        if especie not in cartera and op['Comprobante'] == 'VENTA':
            print(f"ADVERTENCIA: Se intentó vender {cantidad} de {especie}, pero no hay registro de compra. Se omitirá.")
            continue

        elif especie not in cartera:
            cartera[especie] = {'cantidad_total': 0.0, 'costo_total': 0.0}

        # Si es una COMPRA
        if op['Comprobante'] == 'COMPRA NORMAL':
            cartera[especie]['cantidad_total'] += abs(cantidad)
            cartera[especie]['costo_total'] += abs(importe) # El importe de compra es negativo
            print(f"Compra: {cantidad:.2f} de {especie} a ${precio_op:.2f}")


        # Si es una VENTA
        elif op['Comprobante'] == 'VENTA':
            if abs(cartera[especie]['cantidad_total']) < abs(cantidad):
                print(f"ADVERTENCIA: Se intentó vender {cantidad} de {especie}, pero solo hay {cartera[especie]['cantidad_total']} en cartera. Se omitirá.")
                cartera.pop(especie, None)
                print(f"Se eliminó {especie} de la cartera por falta de cantidad suficiente.")
                continue

            # Calcular el Precio Promedio de Compra (PPC) al momento de la venta
            if cartera[especie]['cantidad_total'] > 0:
                ppc = cartera[especie]['costo_total'] / cartera[especie]['cantidad_total']
            else:
                ppc = 0 # Evitar división por cero

            # Calcular el costo de los activos vendidos
            costo_de_venta = ppc * cantidad
            # La ganancia es el importe de la venta (positivo) menos el costo
            ganancia_perdida = abs(importe) - abs(costo_de_venta)

            print(f"Venta: {cantidad:.2f} de {especie} a ${precio_op:.2f}. PPC: ${ppc:.2f}. Resultado: ${ganancia_perdida:.2f}")

            # Registrar el resultado de la operación
            resultados.append({
                'Fecha Venta': op['Operado'].date(),
                'Activo': especie,
                'Cantidad Vendida': cantidad,
                'Precio Venta': precio_op,
                'Precio Promedio Compra (PPC)': ppc,
                'Costo Total Venta': abs(costo_de_venta),
                'Ganancia/Perdida ($)': ganancia_perdida
            })

            # Actualizar la cartera después de la venta
            cartera[especie]['cantidad_total'] -= cantidad
            cartera[especie]['costo_total'] -= costo_de_venta

            # Si se vendió todo, se puede resetear el costo para evitar errores de flotantes
            if cartera[especie]['cantidad_total'] < 1e-9: # Un número muy pequeño
                cartera[especie]['cantidad_total'] = 0
                cartera[especie]['costo_total'] = 0


    print("\n¡Cálculo finalizado!")
    return pd.DataFrame(resultados)

# --- EJECUCIÓN DEL SCRIPT ---
# Reemplaza 'ruta/a/tu/archivo.csv' con la ruta real de tu archivo
# Como subiste el archivo, podemos usar su nombre directamente.
nombre_archivo = '/home/jovyan/data/cuenta_corriente_historic.csv'
resultados_df = calculat_profit_and_loss(nombre_archivo)

# Mostrar el DataFrame con los resultados
if isinstance(resultados_df, pd.DataFrame) and not resultados_df.empty:
    print("\n--- Resumen de Ganancias y Pérdidas Realizadas ---")
    # Formatear los números para mejor lectura
    pd.options.display.float_format = '{:,.2f}'.format
    print(resultados_df.to_string()) # Usamos to_string para ver todas las filas y columnas
else:
    print("\nNo se encontraron operaciones de venta para calcular o hubo un error.")
    if not isinstance(resultados_df, pd.DataFrame):
        print(resultados_df) # Muestra el mensaje de error

Procesando operaciones...
ADVERTENCIA: Se intentó vender -10.0 de ERIC, pero no hay registro de compra. Se omitirá.
Compra: 20.00 de BPAT a $1298.05
Compra: 50.00 de FIPL a $338.86
ADVERTENCIA: Se intentó vender -2.0 de PAAS, pero no hay registro de compra. Se omitirá.
ADVERTENCIA: Se intentó vender -8.0 de SAMI, pero no hay registro de compra. Se omitirá.
Compra: 14.00 de AUSO a $2860.94
Compra: 2.00 de MIRG a $15508.08
Compra: 11.00 de CEPU a $1127.86
Compra: 13.00 de PAMP a $2044.25
Compra: 15.00 de DGCU2 a $1208.42
Compra: 6.00 de CVH a $4818.58
Compra: 18.00 de TRAN a $1386.16
Compra: 2.00 de GGAL a $3974.76
Compra: 1.00 de MIRG a $18126.32
Compra: 5.00 de CVH a $5105.58
Compra: 25.00 de DGCU2 a $1223.53
Compra: 7.00 de PAMP a $2410.30
Compra: 10.00 de AUSO a $3055.29
Compra: 2.00 de PAAS a $8761.06
Venta: -20.00 de BPAT a $2839.93. PPC: $1298.05. Resultado: $30837.65
Compra: 100.00 de FIPL a $304.12
Compra: 9.00 de CEPU a $1206.41
Compra: 9.00 de GGAL a $4330.18
Compra: 10.00 de 

In [3]:
resultados_df.to_csv('/home/jovyan/data/resultados_ganancias_perdidas_historic.csv', index=False)